In [1]:
%pip install -q numpy pandas matplotlib tensorflow scikit-learn setuptools keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [2]:
#run once
import os
os.chdir('../')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [ ]:
#Data Setup
data = pd.read_csv('Data\Building_Data_Preprocessed.csv')

data = data.drop('BuildingName', axis = 1)
data = data.drop('Address', axis = 1)

X = data.drop('GHGEmissionsIntensity', axis = 1)
y = data['GHGEmissionsIntensity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#train_tensor = tf.convert_to_tensor(X_train, dtype = tf.float32)
#valid_tensor = tf.convert_to_tensor(X_test, dtype = tf.float32)

#train_labels = tf.convert_to_tensor(y_train, dtype = tf.float32)
#valid_labels = tf.convert_to_tensor(y_test, dtype = tf.float32)

<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:2: SyntaxWarning: invalid escape sequence '\B'
C:\Users\jfsal\AppData\Local\Temp\ipykernel_8996\1779928584.py:2: SyntaxWarning: invalid escape sequence '\B'
  data = pd.read_csv('Data\Building_Data_Preprocessed.csv')


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband
import kerastuner
from tensorflow.keras.metrics import RootMeanSquaredError


class GHGHyperModel(HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Input(shape=(X_train.shape[1],)))
        model.add(layers.BatchNormalization())

        for i in range(hp.Int('num_layers', 1, 5)):
            units = hp.Int('units_' + str(i), 
                           min_value = 64,
                           max_value = 1028,
                           step = 64)
            
            model.add(layers.Dense(units=units, activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(hp.Float('dropout_' + str(i),
                                              min_value=0.2,
                                              max_value=0.5,
                                              step=0.1)))
        model.add(layers.Dense(1, activation='linear'))

        model.compile(
            optimizer=keras.optimizers.Adam(
                learning_rate=hp.Float('learning_rate', 
                                       min_value=1e-5,
                                       max_value=1e-3,
                                       sampling='LOG')
            ),
            loss='mse',
            metrics=['mean_absolute_error', RootMeanSquaredError()]
        )
        return model

from keras_tuner import RandomSearch

tuner = RandomSearch(
    GHGHyperModel(),
    objective='val_root_mean_squared_error',
    max_trials=100,  # Number of different hyperparameter combinations to try
    executions_per_trial=1,  # Number of models to train with each hyperparameter combination
    directory='Code/GHGNNTuning',
    project_name='GHG Emissions Prediction',
    
)

#********************#
MY_PATIENCE = 10
MY_EPOCHS = 100
MY_MIN_DELTA = 0.01
#********************#

In [ ]:
tuner.search(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs = MY_EPOCHS,
    callbacks = [keras.callbacks.EarlyStopping(patience=MY_PATIENCE, min_delta=MY_MIN_DELTA, restore_best_weights=True)], #prevent overfitting
    verbose = 1
)

best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparams = tuner.get_best_hyperparameters(num_trials=1)[0]

best_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error', RootMeanSquaredError()])
best_model.evaluate(X_test, y_test)

Trial 24 Complete [00h 00m 09s]
val_root_mean_squared_error: 2.6961593627929688

Best val_root_mean_squared_error So Far: 2.6942975521087646
Total elapsed time: 00h 05m 37s

Search: Running Trial #25

Value             |Best Value So Far |Hyperparameter
5                 |4                 |num_layers
832               |384               |units_0
0.2               |0.3               |dropout_0
1.1079e-05        |0.00091609        |learning_rate
576               |960               |units_1
0.4               |0.4               |dropout_1
128               |192               |units_2
0.2               |0.2               |dropout_2
256               |896               |units_3
0.4               |0.3               |dropout_3
256               |192               |units_4
0.2               |0.2               |dropout_4

Epoch 1/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 4.2399 - mean_absolute_error: 1.0866 - root_mean_squared_error: 2.0535 - val_loss: 8.8578 - val_mean_absolute_error